<a href="https://colab.research.google.com/github/marcocmcmelo/projetos_python/blob/main/ProjetoFinal_ETL_equipe4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Parte inicial onde são importadas e carrgadas as bibliotecas necessárias.

In [ ]:
!pip install mysql-connector-python

In [ ]:
!pip install google-cloud-storage

In [ ]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

Trecho de configuração e conexão com banco de dados.

In [ ]:
# ip que será usado na configuração de rede do sql cloud
!curl ipecho.net/plain

In [ ]:
# Configurações do Banco de Dados
host = 'caminho do banco de dados'
user = 'usuário do banco de dados'
password = 'sehna do banco de dados'
database = 'nome do abnco de dados'
porta = 'porta do banco de dados'

Trecho que insere os dados do DataFrame para o banco de dados no Cloud

In [ ]:
# Caminho da planilha
arquivo_excel = "caminho completo onde está os dados brutos"
nome_planilha = "se for planilha excel informar o nome da palniha"  # Nome da aba que será lida

# Nome da tabela no banco de dados
nome_tabela = "nome tabela do banco de dados"

def transf_dados_tabela():
    try:
        # Ler a planilha do Excel
        print("Lendo as informações da planilha...")

        # Criado a variavel repositorio que vai guardar as informações lidas
        #e que serão gravadas no banco da nuvem
        repositorio = pd.read_excel(arquivo_excel, sheet_name=nome_planilha)

        # Efetuando a conexão com o banco de dados
        print("Efetuando a conexão com o banco de dados...")

        # create_engine gerencia as conexões do Python com o banco de dados
        engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{porta}/{database}")

        # Enviar os dados para o banco de dados
        print(f"Enviando os dados para a tabela '{nome_tabela}'...")

        # nesse trecho o método do pandas to.sql exporta os dados do DataFrame para o banco de dados
        repositorio.to_sql(nome_tabela, con=engine, if_exists='replace', index=False)

        print("Dados transferidos com sucesso!")
    except Exception as e: # cláusula try e exception para tratamento de algum erro
        print("Erro ao transferir os dados:", e)

# Executar a função
transf_dados_tabela()

In [ ]:
# Conexão com o banco de dados e consulta a tabela
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
)
cursor = connection.cursor()
query = '''
SELECT * FROM atendimento_chamados;
'''
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=cursor.column_names)
cursor.close()
connection.close()

Transformação e limpeza dos dados
* Renomeando colunas
* Verificação de campos nulos
* Update no banco para correção de valores
* Verificação de linhas duplicadas
* Criação de colunas necessárias ao DataFrame
* Filtros e agregações








In [ ]:
df.head()

In [ ]:
#criação variavel para guardar uma cópia do DataFrame Original
df_original = df.copy()

In [ ]:
# Renomeando as colunas
df.rename(columns={
    'ID do Chamado': 'id_chamados',
    'Data de Abertura': 'data_inicio',
    'Categoria do Chamado': 'categoria_chamados',
    'Nível de Suporte': 'niveis_suporte',
    'Status do Chamado': 'status_chamados',
    'Tempo de Resposta (horas)': 'tempos_resposta',
    'Tempo de Resolução (horas)':'tempos_resolucao',
    'Nível de Satisfação (estrelas)':'niveis_satisfacao',
    'Descrição':'descricao_atendimento'
}, inplace=True)

display(df)

In [ ]:
# Conexão ao banco de dados e da update na coluna tempo_resolução
# corrigindo para ficar com uma casa decimal
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
)

try:
    cursor = connection.cursor()

    # update para arredondar a coluna para 1 casa decimal
    query = '''
    UPDATE atendimento_chamados
    SET tempo_resolucao = ROUND(tempo_resolucao, 1);
    '''
    cursor.execute(query)

    # commit grava as informações no banco de dados
    connection.commit()
    # rowcount faz a contagem das linhas alteradas nesse update
    print(f"{cursor.rowcount} linha(s) atualizada(s).")

except mysql.connector.Error as err:
    print(f"Erro ao executar a consulta: {err}")
finally:

    cursor.close()
    connection.close()

933 linha(s) atualizada(s).


In [ ]:
# Conexão ao banco de dados e da update na coluna tempo_resposta
# corrigindo para ficar com uma casa decimal
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
)

try:
    cursor = connection.cursor()

    # update para arredondar a coluna para 1 casa decimal
    query = '''
    UPDATE atendimento_chamados
    SET tempo_resposta = ROUND(tempo_resposta, 1);
    '''
    cursor.execute(query)

    # commit grava as informações no banco
    connection.commit()
    # rowcount faz a contagem das linhas alteradas nesse update
    print(f"{cursor.rowcount} linha(s) atualizada(s).")

except mysql.connector.Error as err:
    print(f"Erro ao executar a consulta: {err}")
finally:

    cursor.close()
    connection.close()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# Remover linhas vazias
df = df.dropna(how='all')

In [ ]:
# Trás todas as linhas duplicadas (keep=False garante que todas as linhas de mesmo id sejam consideradas duplicadas)
duplicados = df[df.duplicated(subset='id_chamados', keep=False)]
display(duplicados)

,id_chamados,data_inicio,categoria_chamados,niveis_suporte,status_chamados,tempos_resposta,tempos_resolucao,niveis_satisfacao,descricao_atendimento


In [ ]:
# inplace=True -> alterando o df original
df.set_index('id_chamados', inplace=True)
display(df)

In [ ]:
# Contagem das quantidades por categorias de chamados
agregacao_qt_categoria = df['categoria_chamados'].value_counts()
display(agregacao_qt_categoria)

In [ ]:
# Gera um dataframe com as chamadas de categoria 'Hardware'
df_hardware = df[df['categoria_chamados'] == 'Hardware']
display(df_hardware)


In [ ]:
# Gera um dataframe com as chamadas de categoria 'Rede'
df_rede = df[df['categoria_chamados'] == 'Rede']
display(df_rede)

In [ ]:
# Gera um dataframe com as chamadas de categoria 'Acessos'
df_acessos = df[df['categoria_chamados'] == 'Acessos']
display(df_acessos)

In [ ]:
# Gera um dataframe com as chamadas de categoria 'Software'
df_software = df[df['categoria_chamados'] == 'Software']
display(df_software)

In [ ]:
# Gera um dataframe com as chamadas de categoria 'Impressoras'
df_impressoras = df[df['categoria_chamados'] == 'Impressoras']
display(df_impressoras)

In [ ]:
# Gera um dataframe com as chamadas de categoria 'Telefonia'
df_telefonia = df[df['categoria_chamados'] == 'Telefonia']
display(df_telefonia)

In [ ]:
# Contagem das quantidades por status dos chamados
agregacao_qt_status = df['status_chamados'].value_counts()
display(agregacao_qt_status)

In [ ]:
# Contagem das quantidades por nível de suporte
agregacao_qt_suporte = df['niveis_suporte'].value_counts()
display(agregacao_qt_suporte)

In [ ]:
# Contagem das quantidades por nível de satisfação
agregacao_qt_satisfacao = df['niveis_satisfacao'].value_counts()
display(agregacao_qt_satisfacao)

,count
niveis_satisfacao,
4,210
5,208
2,199
3,197
1,186


In [ ]:
# Contagem das quantidades por descrição do atendimento
agregacao_qt_atendimento = df['descricao_atendimento'].value_counts()
display(agregacao_qt_atendimento)

,count
descricao_atendimento,
Equipamento não liga,121
Telefone sem linha,109
Troca de cartucho de impressora,108
Problema na conexão Wi-Fi,108
Hardware defeituoso,106
Solicitação de manutenção preventiva,99
Configuração de novo equipamento,95
Solicitação de acesso a rede,94
Atualização de software necessária,84


In [ ]:
#Cálculo da média da coluna tempos_resposta
media_resposta = round(df['tempos_resposta'].mean(),1)
print(media_resposta)


12.2


In [ ]:
# Geração das medidas descritivas do tempo resposta resposta
media = df['tempos_resposta'].mean()
mediana = df['tempos_resposta'].median()
moda = df['tempos_resposta'].mode()[0]
dp = df['tempos_resposta'].std()
var = df['tempos_resposta'].var()
q1 = df['tempos_resposta'].quantile(0.25)
q2 = df['tempos_resposta'].quantile(0.50)
q3 = df['tempos_resposta'].quantile(0.75)

print('Média: ',media)
print('Mediana: ',mediana)
print('Moda: ',moda)
print('Desvio Padrão: ',dp)
print('Variância: ',var)
print('Q1: ',q1)
print('Q2: ',q2)
print('Q3: ', q3)

Média:  12.184099999999999
Mediana:  11.95
Moda:  1.9
Desvio Padrão:  6.873151926693269
Variância:  47.24021740740739
Q1:  6.4
Q2:  11.95
Q3:  18.2


In [ ]:
# Convertendo a coluna tempos_resolucao que está com object em numerico
# e efetuando a cálculo da média
df['tempos_resolucao'] = pd.to_numeric(df['tempos_resolucao'])
media_resolucao = round(df['tempos_resolucao'].mean(),1)
print(media_resolucao)


45.3


In [ ]:
# Geração das medidas descritivas do tempo de resolução
media = df['tempos_resolucao'].mean()
mediana = df['tempos_resolucao'].median()
moda = df['tempos_resolucao'].mode()[0]
dp = df['tempos_resolucao'].std()
var = df['tempos_resolucao'].var()
q1 = df['tempos_resolucao'].quantile(0.25)
q2 = df['tempos_resolucao'].quantile(0.50)
q3 = df['tempos_resolucao'].quantile(0.75)

print('Média: ',media)
print('Mediana: ',mediana)
print('Moda: ',moda)
print('Desvio Padrão: ',dp)
print('Variância: ',var)
print('Q1: ',q1)
print('Q2: ',q2)
print('Q3: ', q3)

Média:  45.33220000000001
Mediana:  45.6
Moda:  67.9
Desvio Padrão:  22.146217693279322
Variância:  490.4549581181181
Q1:  27.65
Q2:  45.6
Q3:  63.525


In [ ]:
#função para criar uma coluna que descreva os niveis de satisfação
def descrever_satisfacao(niveis_satisfacao):
    if niveis_satisfacao == 1:
        return 'Pessimo'
    elif niveis_satisfacao == 2:
        return 'Ruim'
    elif niveis_satisfacao == 3:
        return 'Regular'
    elif niveis_satisfacao == 4:
        return 'Bom'
    else:
        return 'Otimo'

# Criando uma nova coluna cdescreção com base na coluna nivel satisfaão
df['descricao_satisfacao'] = df['niveis_satisfacao'].apply(descrever_satisfacao)

display(df)

,data_inicio,categoria_chamados,niveis_suporte,status_chamados,tempos_resposta,tempos_resolucao,niveis_satisfacao,descricao_atendimento,descricao_satisfacao
id_chamados,,,,,,,,,
CH000001,2023-02-19,Hardware,N1,Resolvido,18.4,36.7,5,Telefone sem linha,Otimo
CH000002,2023-07-24,Acessos,N1,Resolvido,9.7,44.2,5,Erro ao acessar sistema,Otimo
CH000003,2023-06-30,Acessos,N1,Resolvido,18.0,39.7,3,Solicitação de acesso a rede,Regular
CH000004,2024-05-04,Software,N3,Resolvido,4.7,17.5,2,Equipamento não liga,Ruim
CH000005,2023-12-26,Impressoras,N1,Cancelado,11.8,67.9,4,Solicitação de manutenção preventiva,Bom
...,...,...,...,...,...,...,...,...,...
CH000996,2024-09-20,Rede,N1,Resolvido,7.1,67.0,5,Problema na conexão Wi-Fi,Otimo
CH000997,2024-05-07,Acessos,N1,Aberto,2.7,4.0,2,Atualização de software necessária,Ruim
CH000998,2023-11-17,Hardware,N1,Em andamento,20.7,34.3,4,Telefone sem linha,Bom


In [ ]:
#função para criar uma coluna que descreva os niveis de suporte
def detalhar_suporte(niveis_suporte):
    if niveis_suporte == 'N1':
        return 'Basico'
    elif niveis_suporte == 'N2':
        return 'Avancado'
    elif niveis_suporte == 'N3':
        return 'Especialista'
    else:
        return 'Externo'

# Criando uma nova coluna de descrição com base na coluna nivel suporte
df['descricao_suporte'] = df['niveis_suporte'].apply(detalhar_suporte)

display(df)

,data_inicio,categoria_chamados,niveis_suporte,status_chamados,tempos_resposta,tempos_resolucao,niveis_satisfacao,descricao_atendimento,descricao_satisfacao,descricao_suporte
id_chamados,,,,,,,,,,
CH000001,2023-02-19,Hardware,N1,Resolvido,18.4,36.7,5,Telefone sem linha,Otimo,Basico
CH000002,2023-07-24,Acessos,N1,Resolvido,9.7,44.2,5,Erro ao acessar sistema,Otimo,Basico
CH000003,2023-06-30,Acessos,N1,Resolvido,18.0,39.7,3,Solicitação de acesso a rede,Regular,Basico
CH000004,2024-05-04,Software,N3,Resolvido,4.7,17.5,2,Equipamento não liga,Ruim,Especialista
CH000005,2023-12-26,Impressoras,N1,Cancelado,11.8,67.9,4,Solicitação de manutenção preventiva,Bom,Basico
...,...,...,...,...,...,...,...,...,...,...
CH000996,2024-09-20,Rede,N1,Resolvido,7.1,67.0,5,Problema na conexão Wi-Fi,Otimo,Basico
CH000997,2024-05-07,Acessos,N1,Aberto,2.7,4.0,2,Atualização de software necessária,Ruim,Basico
CH000998,2023-11-17,Hardware,N1,Em andamento,20.7,34.3,4,Telefone sem linha,Bom,Basico


Trecho que efetua as exportações do DataFrame tratado para o BigQuery e para Cloud Storage

In [ ]:
# exportando o DataFrame tratado
df.to_excel('atendimento_tratado.xlsx', index=False)

In [ ]:
# Autenticação rápida e segura sem uso de senhas
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'nome do projeto no cloud'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.cloud import storage

# Definições
arquivo = 'nome do arquivo tratado'

# Gerando o arquivo CSV no Colab
df.to_csv(arquivo, index=False)

# Definição do bucket e do cliente do Google Cloud
bucket_name = 'dnome do bucket no cloud'
client = storage.Client()
bucket = client.bucket(bucket_name)

# Caminho do conjunto no storage
destino_bucket = 'atendimento-tratados/' + arquivo

# Enviando binários do arquivo para o bucket
blob = bucket.blob(destino_bucket)
blob.upload_from_filename(arquivo)

print(f'Arquivo {arquivo} enviado para {destino_bucket}.')

Arquivo atendimento_tratado.xlsx enviado para atendimento-tratados/atendimento_tratado.xlsx.


In [ ]:
df.head()

,id_chamados,data_inicio,categoria_chamados,niveis_suporte,status_chamados,tempos_resposta,tempos_resolucao,niveis_satisfacao,descricao_atendimento,descricao_satisfacao,descricao_suporte
0,CH000001,2023-02-19,Hardware,N1,Resolvido,18.4,36.7,5,Telefone sem linha,Otimo,Basico
1,CH000002,2023-07-24,Acessos,N1,Resolvido,9.7,44.2,5,Erro ao acessar sistema,Otimo,Basico
2,CH000003,2023-06-30,Acessos,N1,Resolvido,18.0,39.7,3,Solicitação de acesso a rede,Regular,Basico
3,CH000004,2024-05-04,Software,N3,Resolvido,4.7,17.5,2,Equipamento não liga,Ruim,Especialista
4,CH000005,2023-12-26,Impressoras,N1,Cancelado,11.8,67.9,4,Solicitação de manutenção preventiva,Bom,Basico


In [ ]:
from google.colab import auth

# Autenticar a entrada sem uso de senhas
auth.authenticate_user()

In [ ]:
# id do projeto que você vai acessar
project_id = 'nome do projeto no cloud'

# A string é formada por project_id.dataset.tabela
table_id = f'{project_id}.atendimento_embraer.atendimento_tratado'

# Manda para a big query o dataframe, e caso já exista, substitui
df.to_gbq(table_id, project_id=project_id, if_exists='replace')

<ipython-input-36-705533327bfc>:10: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(table_id, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 6512.89it/s]
